<a href="https://colab.research.google.com/github/shwoa/Capstone/blob/main/PyCaret_%EC%B5%9C%EB%B9%88%EA%B0%92%EB%8C%80%EC%B2%B4_MIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 모델성능비교_PCC_테이블_gwas.csv to 모델성능비교_PCC_테이블_gwas.csv


In [ ]:
!pip install catboost --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install pycaret[full] --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 14.0 MB/s eta 0:0

# ** < mir > pcc, 전체snp 포함**


1.   토마토_표현형.CSV
2.   유전형인코딩_최빈값.csv
3.   MutualInfo_SNP() _100,500, 1000, 3000, 5000, 10000



In [ ]:
from pycaret.regression import setup, compare_models, predict_model, pull
from scipy.stats import pearsonr
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 1. 데이터 로드
geno_df = pd.read_csv("유전형인코딩_최빈값.csv", index_col=0)
pheno_df = pd.read_csv("토마토_표현형.CSV", index_col=0, encoding="cp949")

# 2. SNP 조건
snp_files = {
    "Top 100": "MutualInfo_SNP(100).csv",
    "Top 500": "MutualInfo_SNP(500).csv",
    "Top 1000": "MutualInfo_SNP(1000).csv",
    "Top 3000": "MutualInfo_SNP(3000).csv",
    "Top 5000": "MutualInfo_SNP(5000).csv",
    "Top 10000": "MutualInfo_SNP(10000).csv",
}

results = []

for label, file_path in snp_files.items():
    if label == "전체 SNP":
        all_snps = geno_df.index.tolist()
        traits = pheno_df.columns.tolist()
    else:
        snp_df = pd.read_csv(file_path)
        traits = snp_df["표현형"].unique()

    for trait in traits:
        try:
            if label == "전체 SNP":
                top_snps = all_snps
            else:
                top_snps = snp_df[snp_df["표현형"] == trait]["SNP"].tolist()

            geno_subset = geno_df.loc[top_snps].T
            geno_subset["target"] = pheno_df[trait]

            # PyCaret 설정
            setup(data=geno_subset, target="target", session_id=42,verbose=False)

            # compare_models: 테이블 출력
            best_models = compare_models(include=["rf", "xgboost", "lightgbm", "catboost"], n_select=4)
            result_df = pull()
            display(result_df)  # 성능 비교 테이블

            # 모델별 PCC 수동 계산 후 result_df에 붙이기
            pcc_list = []
            for model in best_models:
                pred_df = predict_model(model)
                pcc, _ = pearsonr(pred_df["target"], pred_df["prediction_label"])
                pcc_list.append(pcc)

            # PCC 컬럼 추가
            result_df["PCC"] = pcc_list
            result_df["SNP 개수"] = label
            result_df["표현형"] = trait

            results.append(result_df)

        except Exception as e:
            print(f"⚠️ 오류 발생 - 표현형: {trait}, SNP: {label}, 원인: {e}")
            continue

# 3. 결과 저장
if results:
    final_df = pd.concat(results, ignore_index=True)
    final_df.to_csv("모델성능비교_PCC_테이블.csv", index=False)
    print("✅ 결과 저장 완료 → 모델성능비교_PCC_테이블.csv")
    display(final_df.head(10))
else:
    print("❌ 저장할 결과가 없습니다.")


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,28.5812,1590.6010,38.9917,0.4117,0.5286,0.5976,1.9120
rf,Random Forest Regressor,29.0136,1701.5729,40.3269,0.3703,0.5384,0.6032,0.9560
lightgbm,Light Gradient Boosting Machine,30.5919,1870.7550,41.9530,0.3073,0.5827,0.5897,0.2370
xgboost,Extreme Gradient Boosting,31.0962,1972.2258,43.5290,0.2242,0.5980,0.5719,0.1820


Processing:   0%|          | 0/24 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,28.5812,1590.6010,38.9917,0.4117,0.5286,0.5976,1.912
rf,Random Forest Regressor,29.0136,1701.5729,40.3269,0.3703,0.5384,0.6032,0.956
lightgbm,Light Gradient Boosting Machine,30.5919,1870.7550,41.9530,0.3073,0.5827,0.5897,0.237
xgboost,Extreme Gradient Boosting,31.0962,1972.2258,43.5290,0.2242,0.5980,0.5719,0.182


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,34.8166,2198.1370,46.8843,0.1640,0.7151,0.9603


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,35.4511,2435.9782,49.3556,0.0736,0.7597,1.0542


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.0762,0.0089,0.0943,0.3639,0.0589,0.1342


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0749,0.0088,0.0938,0.3703,0.0591,0.1337


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.0801,0.0102,0.1007,0.2742,0.0638,0.1448


✅ 결과 저장 완료 → 모델성능비교_PCC_테이블.csv


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),PCC,SNP 개수,표현형
0,CatBoost Regressor,28.5812,1590.6010,38.9917,0.4117,0.5286,0.5976,1.912,0.593423,Top 100,과중 (g)
1,Random Forest Regressor,29.0136,1701.5729,40.3269,0.3703,0.5384,0.6032,0.956,0.566715,Top 100,과중 (g)
2,Light Gradient Boosting Machine,30.5919,1870.7550,41.9530,0.3073,0.5827,0.5897,0.237,0.559208,Top 100,과중 (g)
3,Extreme Gradient Boosting,31.0962,1972.2258,43.5290,0.2242,0.5980,0.5719,0.182,0.510424,Top 100,과중 (g)
4,Light Gradient Boosting Machine,5.7569,70.3503,7.8234,0.4187,0.1653,0.1317,0.118,0.737546,Top 100,과장 (mm)
5,CatBoost Regressor,5.8252,72.2756,7.9538,0.4103,0.1676,0.1334,2.155,0.782819,Top 100,과장 (mm)
6,Random Forest Regressor,6.2343,80.9922,8.5433,0.2668,0.1765,0.1416,0.295,0.740827,Top 100,과장 (mm)
7,Extreme Gradient Boosting,6.7734,97.2366,9.5003,0.0412,0.1987,0.1545,0.153,0.695252,Top 100,과장 (mm)
8,Random Forest Regressor,8.0661,130.1884,11.0927,0.3468,0.2220,0.1772,0.218,0.657593,Top 100,과폭 (mm)
9,Light Gradient Boosting Machine,8.6932,136.3344,11.4436,0.3379,0.2267,0.1894,0.088,0.633080,Top 100,과폭 (mm)


In [ ]:
import pandas as pd

# 1. 기존 결과 불러오기
df = pd.read_csv("모델성능비교_PCC_테이블.csv")

# 2. 표현형 + SNP 수별 R² 최고 모델만 추출
ranking_df = df.loc[df.groupby(["표현형", "SNP 개수"])["R2"].idxmax()]

# 3. 결과 저장
ranking_df.to_csv("모델성능_100,500,1000,3000,5000,100000랭킹요약.csv", index=False)

# 4. 미리보기
ranking_df.head()


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),PCC,SNP 개수,표현형
20,Random Forest Regressor,0.0804,0.0107,0.1003,0.2806,0.0624,0.1389,0.285,0.667563,Top 100,과실경도 (kg)
68,CatBoost Regressor,0.0748,0.0097,0.0962,0.3360,0.0599,0.1293,19.681,0.615899,Top 1000,과실경도 (kg)
140,Random Forest Regressor,0.0767,0.0100,0.0969,0.3320,0.0603,0.1339,16.094,0.658188,Top 10000,과실경도 (kg)
92,CatBoost Regressor,0.0762,0.0099,0.0969,0.3329,0.0603,0.1322,56.757,0.668416,Top 3000,과실경도 (kg)
44,Random Forest Regressor,0.0771,0.0104,0.0990,0.3021,0.0616,0.1343,1.015,0.660243,Top 500,과실경도 (kg)


In [ ]:
# 📌 Colab 전용 성능 요약 코드 (R² 기준 요약 3종)

import pandas as pd

# # 👉 파일 업로드
# from google.colab import files
# uploaded = files.upload()  # 모델성능비교_전체정리.csv 업로드

# 👉 파일 로드
df = pd.read_csv("모델성능비교_PCC_테이블.csv")

# ✅ 1. 표현형별 최고 R² 모델 추출
top_r2_by_trait = df.loc[df.groupby("표현형")["R2"].idxmax()].reset_index(drop=True)

# ✅ 2. SNP 개수별 평균 R² 계산
mean_r2_by_snp = df.groupby("SNP 개수")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# ✅ 3. 모델별 평균 R² 계산
mean_r2_by_model = df.groupby("Model")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# 👉 결과 출력
print("📌 표현형별 최고 R² 요약")
display(top_r2_by_trait)

print("📌 SNP 개수별 평균 R²")
display(mean_r2_by_snp)

print("📌 모델별 평균 R²")
display(mean_r2_by_model)


📌 표현형별 최고 R² 요약


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),PCC,SNP 개수,표현형
0,CatBoost Regressor,0.0748,0.0097,0.0962,0.3360,0.0599,0.1293,19.681,0.615899,Top 1000,과실경도 (kg)
1,Light Gradient Boosting Machine,5.7569,70.3503,7.8234,0.4187,0.1653,0.1317,0.118,0.737546,Top 100,과장 (mm)
2,CatBoost Regressor,25.6246,1311.7937,35.3588,0.5245,0.5089,0.5454,9.352,0.630866,Top 500,과중 (g)
3,Light Gradient Boosting Machine,6.4973,87.8449,9.2961,0.5413,0.1828,0.1373,3.122,0.751458,Top 5000,과폭 (mm)
4,CatBoost Regressor,0.5413,0.5587,0.7358,0.1593,0.1294,0.1219,203.668,0.676126,Top 10000,과피두께 (mm)
5,CatBoost Regressor,0.6207,0.6781,0.8143,0.3676,0.1597,0.1571,65.039,0.853703,Top 3000,당도 (%)


📌 SNP 개수별 평균 R²


,SNP 개수,평균 R²
0,Top 100,0.258529
1,Top 1000,0.245846
2,Top 10000,0.274867
3,Top 3000,0.277308
4,Top 500,0.263721
5,Top 5000,0.288792


📌 모델별 평균 R²


,Model,평균 R²
0,CatBoost Regressor,0.342933
1,Extreme Gradient Boosting,0.149442
2,Light Gradient Boosting Machine,0.272464
3,Random Forest Regressor,0.307869


In [ ]:
# 📌 Colab 전용 성능 요약 코드 (PCC 기준 요약 3종)

import pandas as pd

# # 👉 파일 업로드
# from google.colab import files
# uploaded = files.upload()  # 모델성능비교_전체정리.csv 업로드

# 👉 파일 로드
df = pd.read_csv("모델성능비교_PCC_테이블.csv")

# ✅ 1. 표현형별 최고 R² 모델 추출
top_r2_by_trait = df.loc[df.groupby("표현형")["PCC"].idxmax()].reset_index(drop=True)

# ✅ 2. SNP 개수별 평균 R² 계산
mean_r2_by_snp = df.groupby("SNP 개수")["PCC"].mean().reset_index().rename(columns={"PCC": "평균 PCC"})

# ✅ 3. 모델별 평균 R² 계산
mean_r2_by_model = df.groupby("Model")["PCC"].mean().reset_index().rename(columns={"PCC": "평균 PCC"})

# 👉 결과 출력
print("📌 표현형별 최고 PCC 요약")
display(top_r2_by_trait)

print("📌 SNP 개수별 평균 PCC")
display(mean_r2_by_snp)

print("📌 모델별 평균 PCC")
display(mean_r2_by_model)


📌 표현형별 최고 PCC 요약


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),PCC,SNP 개수,표현형
0,Random Forest Regressor,0.0782,0.0104,0.0989,0.3030,0.0617,0.1362,7.190,0.679691,Top 5000,과실경도 (kg)
1,CatBoost Regressor,5.8252,72.2756,7.9538,0.4103,0.1676,0.1334,2.155,0.782819,Top 100,과장 (mm)
2,Extreme Gradient Boosting,29.3117,2084.2055,43.6025,0.1312,0.5893,0.7453,5.248,0.687491,Top 5000,과중 (g)
3,Light Gradient Boosting Machine,6.4973,87.8449,9.2961,0.5413,0.1828,0.1373,3.122,0.751458,Top 5000,과폭 (mm)
4,Random Forest Regressor,0.5803,0.6294,0.7818,0.0519,0.1386,0.1309,5.635,0.726250,Top 5000,과피두께 (mm)
5,CatBoost Regressor,0.6377,0.7380,0.8441,0.3061,0.1658,0.1621,20.702,0.864799,Top 1000,당도 (%)


📌 SNP 개수별 평균 PCC


,SNP 개수,평균 PCC
0,Top 100,0.680547
1,Top 1000,0.670056
2,Top 10000,0.669699
3,Top 3000,0.687063
4,Top 500,0.684664
5,Top 5000,0.682996


📌 모델별 평균 PCC


,Model,평균 PCC
0,CatBoost Regressor,0.700252
1,Extreme Gradient Boosting,0.635166
2,Light Gradient Boosting Machine,0.675936
3,Random Forest Regressor,0.705330


In [ ]:
# ✅ 4. SNP 개수별 × 모델별 평균 R² 계산
mean_r2_by_snp_and_model = df.groupby(["SNP 개수", "Model"])["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# 👉 결과 출력
print("📌 SNP 개수별 × 모델별 평균 R²")
display(mean_r2_by_snp_and_model)


📌 SNP 개수별 × 모델별 평균 R²


,SNP 개수,Model,평균 R²
0,Top 100,CatBoost Regressor,0.312850
1,Top 100,Extreme Gradient Boosting,0.168033
2,Top 100,Light Gradient Boosting Machine,0.267200
3,Top 100,Random Forest Regressor,0.286033
4,Top 1000,CatBoost Regressor,0.329267
5,Top 1000,Extreme Gradient Boosting,0.110150
6,Top 1000,Light Gradient Boosting Machine,0.256433
7,Top 1000,Random Forest Regressor,0.287533
8,Top 10000,CatBoost Regressor,0.355000
9,Top 10000,Extreme Gradient Boosting,0.152400


In [ ]:
# ✅ 피벗 테이블 형태로 보기 좋게 변환
pivot_r2_by_snp_and_model = mean_r2_by_snp_and_model.pivot(index="SNP 개수", columns="Model", values="평균 R²")

# 👉 결과 출력
print("📌 SNP 개수별 × 모델별 평균 R² (피벗 테이블)")
display(pivot_r2_by_snp_and_model)


📌 SNP 개수별 × 모델별 평균 R² (피벗 테이블)


Model,CatBoost Regressor,Extreme Gradient Boosting,Light Gradient Boosting Machine,Random Forest Regressor
SNP 개수,,,,
Top 100,0.312850,0.168033,0.267200,0.286033
Top 1000,0.329267,0.110150,0.256433,0.287533
Top 10000,0.355000,0.152400,0.275233,0.316833
Top 3000,0.352283,0.175983,0.258033,0.322933
Top 500,0.348217,0.116350,0.287400,0.302917
Top 5000,0.359983,0.173733,0.290483,0.330967
